In [ ]:
import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element, ElementTree
import os
import pandas as pd
pd.set_option('display.max_row', 500)

In [ ]:
# 비어있는 데이터 프레임을 생성
xml_csv = pd.DataFrame(columns = 
                       ['filename',
                        'center_x',
                        'center_y',
                        'width',
                        'height',
                        'angle',
                        'occupied',
                        'x_max',
                        'x_min',
                        'y_max',
                        'y_min']
                      )
xml_csv = xml_csv[['filename',
                    'center_x',
                    'center_y',
                    'width',
                    'height',
                    'angle',
                    'occupied',
                    'x_max',
                    'x_min',
                    'y_max',
                    'y_min']]
xml_csv

,filename,center_x,center_y,width,height,angle,occupied,x_max,x_min,y_max,y_min


In [ ]:
#폴더의 경로
data_dir = './ProjectData/PKLot/'
data_kind = os.listdir('./ProjectData/PKLot/')

#3개의 폴더 총 3번의 반복
for i in range(len(data_kind)):
    a = os.path.join(data_dir, data_kind[i]+'/')
    b = os.path.join(a,'xml'+'/')
    data_xml = os.listdir(b) # xml 폴더에 있는 파일 리스트 출력
    file_name = [fn.split('.')[0] for fn in data_xml] #파일이름만 가져오기 뒤에 .xml제외하고
    
    # 각각의 폴더안에있는 xml파일들의 갯수들을 구하여 그 갯수만큼 반복
    for j in range(len(data_xml)):
        c = os.path.join(b,data_xml[j]) # 각각의 파일의 이름을 구하여 최종 경로 생성
        file_name = data_xml[j].split('.')[0] # 파일 id 저장
        xml = open(c, "r") # read mode로 읽기
        tree = Et.parse(xml) # 파싱함수
        root = tree.getroot() # 루트 요소를 가져옴
        space = root.findall("space") # 루트 요소 하위의 "space" 요소를 모두 가져옴 (리스트 형태) 
        
        # space 리스트의 요소 개수 만큼 반복
        for k in range(len(space)): 
            # 데이터프레임에 추가해줄 데이터의 행 (딕셔너리 형태)를 생성
            data_insert = {
                'filename':0,
                'center_x':0,
                'center_y':0,
                'width':0,
                'height':0,
                'angle':0,
                'occupied':0,
                'x_max':0,
                'x_min':0,
                'y_max':0,
                'y_min':0
            }

            # 주차 공간의 좌표를 저장할 리스트
            x = []
            y = []
            
            rotatedRect = space[k].find("rotatedRect")
            contour = space[k].find("contour")
            point = contour.findall("point") # 주차공간(사각형)의 꼭짓점 좌표들 (x, y) 

            # 주차 공간의 꼭짓점을 저장함
            if len(point) == 0: # 좌표가 없다면
                point = contour.findall("Point")
            for p in range(len(point)): # 꼭짓점의 개수 만큼 반복
                x_ = point[p].attrib['x'] # 꼭짓점 요소의 x 속성을 가져옴
                y_ = point[p].attrib['y'] # 꼭짓점 요소의 y 속성을 가져옴
                x.append(x_)
                y.append(y_)

            # 주차 공간의 센터 좌표를 저장함
            center_x = rotatedRect.find("center").attrib['x'] 
            center_y = rotatedRect.find("center").attrib['y'] 

            # 주차 공간의 길이를 저장함
            width = rotatedRect.find("size").attrib['w'] 
            height = rotatedRect.find("size").attrib['h'] 

            # 주차 공간의 각도를 저장함
            angle = rotatedRect.find("angle").attrib['d']

            # 주차 공간의 빈자리 여부를 저장함, occupied 요소가 없다면 결측지 처리
            try :
                occupied = space[k].attrib['occupied']
            except:
                occupied = 'NA'
            
            # 주차 공간 id를 저장
            id_ = space[k].attrib['id']

            # 주차 공간의 고유 아이디를 저장
            file_ = file_name+'+'+ id_

            # x좌표 y좌표를 넣어준 리스트에서 각각의 최댓값과 최솟값을 뽑아내어 변수에 저장
            x_max = max(x)
            x_min = min(x)
            y_max = max(y)
            y_min = min(y)
            print(file_)

            # 데이터 프레임에 넣어주기 위해 딕셔너리에 각각의 정보들을 저장
            data_insert['filename'] = file_
            data_insert['center_x'] = center_x
            data_insert['center_y'] = center_y
            data_insert['width'] = width
            data_insert['height'] = height
            data_insert['angle'] = angle
            data_insert['occupied'] = occupied
            data_insert['x_max'] = x_max
            data_insert['x_min'] = x_min
            data_insert['y_max'] = y_max
            data_insert['y_min'] = y_min

            #최종적으로 데이터프레임에 딕셔너리 형태의 데이터를 저장
            xml_csv = xml_csv.append(data_insert, ignore_index=True)
# 최종적으로 데이터프레임을 출력
xml_csv

In [ ]:
# na값만 모아놓은 버전 na값을 뺀 버전 그리고 마지막으로 모든 데이터가 들어가있는 데이터프레임을 각각생성
# 그리고 인덱스 초기화 및 파일 이름을 인덱스로 설정
xml_csv_na = xml_csv[xml_csv['occupied'] == 'NA']
xml_csv_no_na = xml_csv[xml_csv.occupied != 'NA']
xml_csv_na = xml_csv_na.reset_index()
xml_csv_na.drop(['index'],axis=1,inplace=True)
xml_csv_no_na = xml_csv_no_na.reset_index()
xml_csv_no_na.drop(['index'],axis=1,inplace=True)
xml_csv = xml_csv.set_index("filename")
xml_csv_na = xml_csv_na.set_index("filename")
xml_csv_no_na = xml_csv_no_na.set_index("filename")

In [ ]:
#각각의 데이터프레임을 csv형태의 파일로 저장
xml_csv_na.to_csv("data_na.csv")
xml_csv_no_na.to_csv("data_no_na.csv")
xml_csv.to_csv("all_data.csv")